In [37]:
import sched, time
import yfinance as yf
import cryptocompare as cp

In [7]:
import requests as r, json
from getpass import getpass

In [5]:
url = 'https://bigdataforall.com/crypto/?'

In [8]:
print('Enter your bitcoin account key')
key = getpass()

Enter your bitcoin account key
········


In [92]:
profile_url = f"{url}key={key}&request=profile"
reset_url =  f"{url}key={key}&request=reset"
buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"

In [139]:
## reset data
response = r.get(reset_url).json()
## set initial params
qty = 0.1  ## qty planning to buy 
sym = "BTC" 
p_qty = 0 ## purchased qty

In [140]:
response

{'username': 'jodie',
 'cash': 100000,
 'shares value': 0,
 'penalty': 0,
 'total': 100000,
 'shares': []}

In [138]:

while True:
    print ("checking balance and bitcoin price...")
    ## get the current BTC price and current account profile
    BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']
    response = r.get(profile_url).json()
    cash = response['cash']
    total = response['total']
    b_asset = total - cash
    
    print(f"Cash = {cash} , Total = {total}, purchased bitcoin value = {b_asset} /// Bitcion price = {BTC}")
    
    if p_qty == 0: ## first time buy -- buy 45%
        qty = cash*0.45/BTC
        buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
        response = r.get(buy_url).json()
        p_qty = p_qty + qty
        b_cost = cash*0.45
        avg_cost = b_cost / p_qty
        print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost < BTC: ## if making money: sell 30% owned bitcoin
        if (b_asset/total) > 0.35:
            qty = b_asset*0.30/BTC
            sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"
            response = r.get(sell_url).json()
            p_qty = p_qty - qty
            b_cost = b_cost- (avg_cost*qty)
            print(f"sold {qty} coins, now total own {p_qty} coins")
        else:  ## if owned bitcoin is less than 35% of the total value, then sell 30% but buy 16% back (to increase MA)
            qty = b_asset*0.3/BTC
            sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"
            response = r.get(sell_url).json()
            p_qty = p_qty - qty
            b_cost = b_cost- (avg_cost*qty)
            print(f"sold {qty} coins, now total own {p_qty} coins")
            ## 15% purchase for adjust MA:
            qty = cash*0.16/BTC
            buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
            response = r.get(buy_url).json()
            p_qty = p_qty + qty
            b_cost = b_cost + (cash*0.16)
            avg_cost = b_cost / p_qty
            print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost > BTC:  ## if not making money: buy 20% cash value bitcoin
        ## but we have to sell 25% bitcions when owned bitcoins is over 78% -- to make sure we always have money to buy: 
        if (b_asset/total) < 0.78:
            qty = cash*0.20/BTC
            buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
            response = r.get(buy_url).json()
            p_qty = p_qty + qty
            b_cost = b_cost + (cash*0.20)
            avg_cost = b_cost / p_qty
            print(f"purchased {qty} coins, now total own {p_qty} coins")
        else: 
            qty = cash*0.20/BTC
            buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
            response = r.get(buy_url).json()
            p_qty = p_qty + qty
            b_cost = b_cost + (cash*0.20)
            avg_cost = b_cost / p_qty
            print(f"purchased {qty} coins, now total own {p_qty} coins")
            qty = b_asset*0.25/BTC
            sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"
            response = r.get(sell_url).json()
            p_qty = p_qty - qty
            b_cost = b_cost- (avg_cost*qty)
            print(f"sold {qty} coins, now total own {p_qty} coins")
        
    else:
        print("no action")
    
    
    time.sleep(32)

checking balance and bitcoin price...
Cash = 100000 , Total = 100000, purchased bitcoin value = 0 /// Bitcion price = 54856.07
purchased 0.8203285434045858 coins, now total own 0.8203285434045858 coins
checking balance and bitcoin price...
Cash = 55013.067833696 , Total = 100028.08804933, purchased bitcoin value = 45015.02021563401 /// Bitcion price = 54867.4
sold 0.24612987064614328 coins, now total own 0.5741986727584425 coins
checking balance and bitcoin price...
Cash = 68518.171993972 , Total = 99992.539205996, purchased bitcoin value = 31474.367212023993 /// Bitcion price = 54884.05
sold 0.1720410604466543 coins, now total own 0.4021576123117882 coins
purchased 0.1872625252527064 coins, now total own 0.5894201375644946 coins
checking balance and bitcoin price...
Cash = 67683.760490384 , Total = 99999.951420446, purchased bitcoin value = 32316.190930062003 /// Bitcion price = 54857.26
purchased 0.24676318317897758 coins, now total own 0.8361833207434722 coins


KeyboardInterrupt: 

In [46]:
# test -- 
BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']

In [47]:
BTC

54691.59